In [14]:
def fetch_data_for_date(date):
    formatted_date = date.strftime('%d-%m-%Y')
    EOD_data = cm.bhav_copy_equities(formatted_date)
    filtered_data_df = EOD_data[EOD_data['SYMBOL'].isin(nifty_500_list['Symbol'])]
    final_data = filtered_data_df[filtered_data_df['SERIES'] == 'EQ']
    final_data.reset_index(drop=True, inplace=True)
    return pd.DataFrame(final_data)

def filter_data_for_common_symbols(df_list):
    common_symbols = set(df_list[0]['SYMBOL'])  # Start with symbols from the first DataFrame
    
    # Find common symbols among all DataFrames
    for df in df_list[1:]:
        common_symbols = common_symbols.intersection(set(df['SYMBOL']))
    
    # Filter each DataFrame to include only common symbols
    filtered_dfs = [df[df['SYMBOL'].isin(common_symbols)] for df in df_list]
    
    return filtered_dfs

# Get today's date
today = datetime(2024, 4, 15)

# Fetch data for today and the previous three days
df = fetch_data_for_date(today)
df1 = fetch_data_for_date(today - timedelta(days=3))
df2 = fetch_data_for_date(today - timedelta(days=5))
df3 = fetch_data_for_date(today - timedelta(days=6))

# Filter data to include only common symbols
df, df1, df2, df3 = filter_data_for_common_symbols([df, df1, df2, df3])

# Display the filtered DataFrames
print("Today's Data:")
print(df)
print("\nYesterday's Data:")
print(df1)
print("\nDay Before Yesterday's Data:")
print(df2)
print("\nThree Days Ago Data:")
print(df3)

Today's Data:
         SYMBOL SERIES      OPEN      HIGH       LOW     CLOSE      LAST  \
0        360ONE     EQ    745.05    756.70    730.00    739.90    739.25   
1       3MINDIA     EQ  29610.00  29610.00  29001.00  29186.30  29151.95   
2      AARTIIND     EQ    740.00    745.50    723.05    735.35    734.15   
3         AAVAS     EQ   1538.20   1559.20   1498.00   1549.95   1550.00   
4           ABB     EQ   6660.00   6729.05   6547.05   6561.05   6573.80   
..          ...    ...       ...       ...       ...       ...       ...   
488        ZEEL     EQ    144.30    145.00    141.20    141.85    141.95   
489  ZENSARTECH     EQ    600.55    604.05    579.95    591.15    592.70   
490   ZFCVINDIA     EQ  14180.20  14400.00  13800.00  13855.65  13843.30   
491      ZOMATO     EQ    188.00    192.40    184.50    188.30    189.50   
492   ZYDUSLIFE     EQ    950.00    962.00    933.50    935.15    935.60   

     PREVCLOSE  TOTTRDQTY     TOTTRDVAL    TIMESTAMP  TOTALTRADES  
0    

In [15]:
def detect_dragonfly_doji(df):
    dragonfly_doji_stocks = []

    for index, row in df.iterrows():
        if abs(row['OPEN']-row['CLOSE'])<0.05*(row['HIGH']-row['LOW']) and 0.05*(min(row['OPEN'],row['CLOSE'])-row['LOW'])> (row['HIGH']-min(row['OPEN'],row['CLOSE'])) and abs(row['HIGH']-max(row['OPEN'],row['CLOSE']))<0.05*(row['HIGH']-row['LOW']):
            dragonfly_doji_stocks.append(row['SYMBOL'])
    return dragonfly_doji_stocks

dragonfly_doji_stocks = detect_dragonfly_doji(df)

print("Stocks in which Dragonfly Doji candlesticks were formed:")
for stock in dragonfly_doji_stocks:
    print(stock)

# Create a DataFrame from the list of stocks
output_df = pd.DataFrame({'Stocks in which Dragonfly Doji candlesticks were formed': dragonfly_doji_stocks})

# Export the DataFrame to an Excel file
output_df.to_excel("output_dragonfly_doji.xlsx", index=False)

def detect_hammer(df, df1, df2,df3):
    hammer_stocks = []

    for index, row in df.iterrows():
        symbol = row['SYMBOL']
        
        # Get corresponding rows from df1 and df2
        row1_prev = df1[df1['SYMBOL'] == symbol].iloc[0]
        row2_prev = df2[df2['SYMBOL'] == symbol].iloc[0]
        row3_prev = df3[df3['SYMBOL'] == symbol].iloc[0]

        # Conditions to check for Hanging Man pattern
        if (row1_prev['CLOSE'] < row1_prev['OPEN']) and \
           (row2_prev['CLOSE'] < row2_prev['OPEN']) and \
           (row3_prev['CLOSE'] < row3_prev['OPEN']) and \
           (row1_prev['CLOSE'] < row3_prev['CLOSE']) and \
           (2 * abs(row['OPEN'] - row['CLOSE']) < (min(row['OPEN'], row['CLOSE']) - row['LOW'])) and \
           (row['HIGH'] - max(row['OPEN'], row['CLOSE']) < 0.05 * (row['HIGH'] - row['LOW'])):
                hammer_stocks.append(symbol)

    return hammer_stocks

# Call the function with your three dataframes
hammer_stocks = detect_hammer(df, df1, df2,df3)

print("Stocks in which Hammer candlesticks were formed:")
for stock in hammer_stocks:
    print(stock)

# Create a DataFrame from the list of stocks
output_df = pd.DataFrame({'Stocks in which Hammer candlesticks were formed': hammer_stocks})

# Export the DataFrame to an Excel file
output_df.to_excel("output_hammer.xlsx", index=False)

def detect_bullish_engulfing(df, df1, df2,df3):
    bullish_engulfing_stocks = []

    for index, row in df.iterrows():
        symbol = row['SYMBOL']
        
        # Get corresponding rows from df1 and df2
        row1_prev = df1[df1['SYMBOL'] == symbol].iloc[0]
        row2_prev = df2[df2['SYMBOL'] == symbol].iloc[0]
        row3_prev = df3[df3['SYMBOL'] == symbol].iloc[0]

        # Conditions to check for bullish engulfing pattern
        if (row1_prev['CLOSE'] < row1_prev['OPEN']) and \
           (row2_prev['CLOSE'] < row2_prev['OPEN']) and \
           (row3_prev['CLOSE'] < row3_prev['OPEN']) and \
           (row1_prev['CLOSE'] < row3_prev['CLOSE']) and \
           (row['OPEN'] < row1_prev['CLOSE']) and \
           (row['CLOSE'] > row1_prev['OPEN']):
                bullish_engulfing_stocks.append(symbol)

    return bullish_engulfing_stocks

# Call the function with your three dataframes
bullish_engulfing_stocks = detect_bullish_engulfing(df, df1, df2,df3)

print("Stocks in which bullish engulfing candlesticks were formed:")
for stock in bullish_engulfing_stocks:
    print(stock)

# Create a DataFrame from the list of stocks
output_df = pd.DataFrame({'Stocks in which bullish engulfing candlesticks were formed': bullish_engulfing_stocks})

# Export the DataFrame to an Excel file
output_df.to_excel("output_bullishengulfing.xlsx", index=False)

def detect_piercing(df, df1, df2,df3):
    piercing_stocks = []

    for index, row in df.iterrows():
        symbol = row['SYMBOL']
        
        # Get corresponding rows from df1 and df2
        row1_prev = df1[df1['SYMBOL'] == symbol].iloc[0]
        row2_prev = df2[df2['SYMBOL'] == symbol].iloc[0]
        row3_prev = df3[df3['SYMBOL'] == symbol].iloc[0]

        # Conditions to check for bullish engulfing pattern
        if (row1_prev['CLOSE'] < row1_prev['OPEN']) and \
           (row['CLOSE'] > row['OPEN']) and \
           (row['OPEN'] < row1_prev['LOW']) and \
           ((abs(row1_prev['OPEN']+row1_prev['CLOSE']))/2 < row['CLOSE']):
                piercing_stocks.append(symbol)

    return piercing_stocks

# Call the function with your three dataframes
piercing_stocks = detect_piercing(df, df1, df2,df3)

print("Stocks in which piercing candlesticks were formed:")
for stock in piercing_stocks:
    print(stock)

# Create a DataFrame from the list of stocks
output_df = pd.DataFrame({'Stocks in which piercing candlesticks were formed': piercing_stocks})

# Export the DataFrame to an Excel file
output_df.to_excel("output_piercing.xlsx", index=False)

def detect_mrngstar(df, df1, df2,df3):
    mrngstar_stocks = []

    for index, row in df.iterrows():
        symbol = row['SYMBOL']
        
        # Get corresponding rows from df1 and df2
        row1_prev = df1[df1['SYMBOL'] == symbol].iloc[0]
        row2_prev = df2[df2['SYMBOL'] == symbol].iloc[0]
        row3_prev = df3[df3['SYMBOL'] == symbol].iloc[0]

        # Conditions to check for Hanging Man pattern
        if (row2_prev['CLOSE'] < row2_prev['OPEN']) and \
           (row3_prev['CLOSE'] < row3_prev['OPEN']) and \
           (row2_prev['CLOSE'] < row3_prev['CLOSE']) and \
           abs((row2_prev['OPEN'] - row2_prev['CLOSE'])) > 0.1*(row2_prev['PREVCLOSE']) and \
           (row1_prev['OPEN'] < row1_prev['PREVCLOSE']) and \
           (abs(row1_prev['OPEN']-row1_prev['CLOSE']) < 0.05*(row1_prev['HIGH']-row1_prev['LOW'])) and\
           row['OPEN'] < row['CLOSE'] and \
           abs(row['OPEN'] - row['CLOSE']) > 0.75 * (abs(row2_prev['OPEN'] - row2_prev['CLOSE'])):
                mrngstar_stocks.append(symbol)

    return mrngstar_stocks

# Call the function with your three dataframes
mrngstar_stocks = detect_mrngstar(df, df1, df2,df3)

print("Stocks in which mrngstar candlesticks were formed:")
for stock in mrngstar_stocks:
    print(stock)

# Create a DataFrame from the list of stocks
output_df = pd.DataFrame({'Stocks in which mrngstar candlesticks were formed': mrngstar_stocks})

# Export the DataFrame to an Excel file
output_df.to_excel("output_mrngstar.xlsx", index=False)

def detect_invertedhammer(df, df1, df2,df3):
    invertedhammer_stocks = []

    for index, row in df.iterrows():
        symbol = row['SYMBOL']
        
        # Get corresponding rows from df1 and df2
        row1_prev = df1[df1['SYMBOL'] == symbol].iloc[0]
        row2_prev = df2[df2['SYMBOL'] == symbol].iloc[0]
        row3_prev = df3[df3['SYMBOL'] == symbol].iloc[0]

        # Conditions to check for Hanging Man pattern
        if (row1_prev['CLOSE'] < row1_prev['OPEN']) and \
           (row2_prev['CLOSE'] < row2_prev['OPEN']) and \
           (row3_prev['CLOSE'] < row3_prev['OPEN']) and \
           (row1_prev['CLOSE'] < row3_prev['CLOSE']):
            if 2 * abs(row['OPEN'] - row['CLOSE']) < (row['HIGH'] - max(row['OPEN'], row['CLOSE'])) and \
               min(row['OPEN'], row['CLOSE']) - row['LOW'] < 0.05 * (row['HIGH'] - row['LOW']):
                invertedhammer_stocks.append(symbol)

    return invertedhammer_stocks

# Call the function with your three dataframes
invertedhammer_stocks = detect_invertedhammer(df, df1, df2,df3)

print("Stocks in which Invertedhammer candlesticks were formed:")
for stock in invertedhammer_stocks:
    print(stock)

# Create a DataFrame from the list of stocks
output_df = pd.DataFrame({'Stocks in which invertedhammer candlesticks were formed': invertedhammer_stocks})

# Export the DataFrame to an Excel file
output_df.to_excel("output_invertedhammer.xlsx", index=False)

def detect_bullish_harami(df, df1, df2,df3):
    bullish_harami_stocks = []

    for index, row in df.iterrows():
        symbol = row['SYMBOL']
        
        # Get corresponding rows from df1 and df2
        row1_prev = df1[df1['SYMBOL'] == symbol].iloc[0]
        row2_prev = df2[df2['SYMBOL'] == symbol].iloc[0]
        row3_prev = df3[df3['SYMBOL'] == symbol].iloc[0]

        # Conditions to check for bullish engulfing pattern
        if (row1_prev['CLOSE'] < row1_prev['OPEN']) and \
           (row2_prev['CLOSE'] < row2_prev['OPEN']) and \
           (row3_prev['CLOSE'] < row3_prev['OPEN']) and \
           (row1_prev['CLOSE'] < row3_prev['CLOSE']) and \
           (row['CLOSE'] > row['OPEN']) and\
           (row['OPEN'] > row1_prev['CLOSE']) and \
           (row['CLOSE'] < row1_prev['OPEN']):
                bullish_harami_stocks.append(symbol)

    return bullish_harami_stocks

# Call the function with your three dataframes
bullish_harami_stocks = detect_bullish_harami(df, df1, df2,df3)

print("Stocks in which bullish harami candlesticks were formed:")
for stock in bullish_harami_stocks:
    print(stock)

# Create a DataFrame from the list of stocks
output_df = pd.DataFrame({'Stocks in which bullish harami candlesticks were formed': bullish_harami_stocks})

# Export the DataFrame to an Excel file
output_df.to_excel("output_bullishharami.xlsx", index=False)

def detect_whitesoldiers(df, df1, df2,df3):
    whitesoldiers_stocks = []

    for index, row in df.iterrows():
        symbol = row['SYMBOL']
        
        # Get corresponding rows from df1 and df2
        row1_prev = df1[df1['SYMBOL'] == symbol].iloc[0]
        row2_prev = df2[df2['SYMBOL'] == symbol].iloc[0]
        row3_prev = df3[df3['SYMBOL'] == symbol].iloc[0]

        # Conditions to check for bullish engulfing pattern
        if (row1_prev['CLOSE'] > row1_prev['OPEN']) and \
           (row2_prev['CLOSE'] > row2_prev['OPEN']) and \
           (row1_prev['CLOSE'] > row2_prev['CLOSE']) and \
           (row['CLOSE'] > row['OPEN']) and\
           (row2_prev['OPEN'])> (row3_prev['OPEN']+row3_prev['CLOSE'])/2 and\
           (row1_prev['OPEN']-row1_prev['CLOSE']) > 0.1*(row1_prev['PREVCLOSE']) and\
           (row1_prev['HIGH']-row1_prev['CLOSE']) <0.01*(row1_prev['HIGH']-row1_prev['LOW']) and\
           (row['HIGH']-row['CLOSE']) <0.01*(row['HIGH']-row['LOW']) :
                whitesoldiers_stocks.append(symbol)

    return whitesoldiers_stocks

# Call the function with your three dataframes
whitesoldiers_stocks = detect_whitesoldiers(df, df1, df2,df3)

print("Stocks in which whitesoldiers candlesticks were formed:")
for stock in whitesoldiers_stocks:
    print(stock)

# Create a DataFrame from the list of stocks
output_df = pd.DataFrame({'Stocks in which whitesoldiers candlesticks were formed': whitesoldiers_stocks})

# Export the DataFrame to an Excel file
output_df.to_excel("output_whitesoldiers.xlsx", index=False)

def detect_bullishca(df, df1, df2,df3):
    bullishca_stocks = []

    for index, row in df.iterrows():
        symbol = row['SYMBOL']
        
        # Get corresponding rows from df1 and df2
        row1_prev = df1[df1['SYMBOL'] == symbol].iloc[0]
        row2_prev = df2[df2['SYMBOL'] == symbol].iloc[0]
        row3_prev = df3[df3['SYMBOL'] == symbol].iloc[0]

        # Conditions to check for bullish engulfing pattern
        if (row1_prev['CLOSE'] < row1_prev['OPEN']) and \
           (row2_prev['CLOSE'] < row2_prev['OPEN']) and \
           (row3_prev['CLOSE'] < row3_prev['OPEN']) and \
           (row1_prev['CLOSE'] < row3_prev['CLOSE']) and \
           (row['CLOSE'] > row['OPEN']) and\
           (row['OPEN'] < row['PREVCLOSE']) and\
           abs(row['CLOSE']-row['PREVCLOSE']) < 0.01*(row['HIGH']-row['LOW'])  :
                bullishca_stocks.append(symbol)

    return bullishca_stocks

# Call the function with your three dataframes
bullishca_stocks = detect_bullishca(df, df1, df2,df3)

print("Stocks in which bullish counterattack candlesticks were formed:")
for stock in bullishca_stocks:
    print(stock)

# Create a DataFrame from the list of stocks
output_df = pd.DataFrame({'Stocks in which bullish counterattack candlesticks were formed': bullishca_stocks})

# Export the DataFrame to an Excel file
output_df.to_excel("output_bullishcounterattack.xlsx", index=False)

def detect_gmarubozu(df):
    gmarubozu_stocks = []

    for index, row in df.iterrows():
        if abs(row['CLOSE']-row['OPEN']) > 0.1*(row['PREVCLOSE']) and\
           row['CLOSE'] > row['OPEN'] and\
           row['HIGH']-row['CLOSE'] < 0.01*(row['HIGH']-row['LOW']) and\
           row['OPEN']-row['LOW'] < 0.01*(row['HIGH']-row['LOW']):
                gmarubozu_stocks.append(row['SYMBOL'])
    return gmarubozu_stocks

# Detect Doji candlesticks for stocks
gmarubozu_stocks = detect_gmarubozu(df)

print("Stocks in which gmarubozu candlesticks were formed:")
for stock in gmarubozu_stocks:
    print(stock)

# Create a DataFrame from the list of stocks
output_df = pd.DataFrame({'Stocks in which green marubozu candlesticks were formed': gmarubozu_stocks})

# Export the DataFrame to an Excel file
output_df.to_excel("output_gmarubozu.xlsx", index=False)

def detect_tweezerbottom(df, df1):
    tweezerbottom_stocks = []

    for index, row in df.iterrows():
        symbol = row['SYMBOL']
        
        # Get corresponding rows from df1 and df2
        row1_prev = df1[df1['SYMBOL'] == symbol].iloc[0]
        row2_prev = df2[df2['SYMBOL'] == symbol].iloc[0]
        row3_prev = df3[df3['SYMBOL'] == symbol].iloc[0]

        # Conditions to check for Hanging Man pattern
        if (row1_prev['CLOSE'] < row1_prev['OPEN']) and \
           (row['CLOSE'] > row['OPEN']) and \
           abs(row['OPEN'] - row['PREVCLOSE']) < 0.01*(row['HIGH']-row['LOW']):
                tweezerbottom_stocks.append(symbol)

    return tweezerbottom_stocks

# Call the function with your three dataframes
tweezerbottom_stocks = detect_tweezerbottom(df, df1)

print("Stocks in which tweezer bottom candlesticks were formed:")
for stock in tweezerbottom_stocks:
    print(stock)

# Create a DataFrame from the list of stocks
output_df = pd.DataFrame({'Stocks in which tweezerbottom candlesticks were formed': tweezerbottom_stocks})

# Export the DataFrame to an Excel file
output_df.to_excel("output_tweezerbottom.xlsx", index=False)

Stocks in which Dragonfly Doji candlesticks were formed:
Stocks in which Hammer candlesticks were formed:
PNCINFRA
Stocks in which bullish engulfing candlesticks were formed:
GLAXO
INTELLECT
JKLAKSHMI
JUBLPHARMA
REDINGTON
Stocks in which piercing candlesticks were formed:
CERA
CGCL
GAIL
GUJGASLTD
IGL
IIFL
INDUSTOWER
INTELLECT
METROBRAND
MGL
MRPL
NMDC
PRESTIGE
REDINGTON
RHIM
RKFORGE
SAFARI
Stocks in which mrngstar candlesticks were formed:
Stocks in which Invertedhammer candlesticks were formed:
ADANIGREEN
Stocks in which bullish harami candlesticks were formed:
ELECON
Stocks in which whitesoldiers candlesticks were formed:
Stocks in which bullish counterattack candlesticks were formed:
GESHIP
Stocks in which gmarubozu candlesticks were formed:
Stocks in which tweezer bottom candlesticks were formed:
CHOLAHLDNG
OIL
SANOFI


In [16]:
# Merge all the individual DataFrames into a single DataFrame
merged_df = pd.concat([
    pd.DataFrame({'Dragonfly Doji': dragonfly_doji_stocks}),
    pd.DataFrame({'Hammer': hammer_stocks}),
    pd.DataFrame({'Bullish Engulfing': bullish_engulfing_stocks}),
    pd.DataFrame({'Piercing': piercing_stocks}),
    pd.DataFrame({'Morning Star': mrngstar_stocks}),
    pd.DataFrame({'Inverted Hammer': invertedhammer_stocks}),
    pd.DataFrame({'Bullish Harami': bullish_harami_stocks}),
    pd.DataFrame({'White Soldiers': whitesoldiers_stocks}),
    pd.DataFrame({'Bullish Counterattack': bullishca_stocks}),
    pd.DataFrame({'Green Marubozu': gmarubozu_stocks}),
    pd.DataFrame({'Tweezer Bottom': tweezerbottom_stocks})
], axis=1)

# Export the merged DataFrame to an Excel file
merged_df.to_excel("output_all_candlestick_patterns.xlsx", index=False)
